In [2]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder('./dataset/train', transform=transform)
test_dataset = ImageFolder('./dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


# Define the step size and gamma (reduction factor)
step_size = 3
gamma = 0.1

# Create a learning rate scheduler
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

num_epochs = 10

for epoch in range(num_epochs):
    # Step the learning rate
    scheduler.step()
    
    model.train()  # Set the model to training mode
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}] Test Accuracy: {accuracy:.2f}% | Learning Rate: {scheduler.get_lr()[0]:.6f}')

torch.save(model.state_dict(), 'cat_recognition_model4.pth')


/opt/homebrew/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/opt/homebrew/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:384: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Epoch [1/10] Test Accuracy: 85.71% | Learning Rate: 0.000100
Epoch [2/10] Test Accuracy: 71.43% | Learning Rate: 0.000100
Epoch [3/10] Test Accuracy: 85.71% | Learning Rate: 0.000001
Epoch [4/10] Test Accuracy: 85.71% | Learning Rate: 0.000010
Epoch [5/10] Test Accuracy: 85.71% | Learning Rate: 0.000010
Epoch [6/10] Test Accuracy: 85.71% | Learning Rate: 0.000000
Epoch [7/10] Test Accuracy: 85.71% | Learning Rate: 0.000001
Epoch [8/10] Test Accuracy: 85.71% | Learning Rate: 0.000001
Epoch [9/10] Test Accuracy: 85.71% | Learning Rate: 0.000000
Epoch [10/10] Test Accuracy: 85.71% | Learning Rate: 0.000000
